In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [ ]:
print(os.listdir("../input"))

In [ ]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    '../input/training2/train2', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)


In [ ]:
#inspired from https://www.tensorflow.org/tutorials/keras/text_classification
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

In [ ]:
#inspired from https://www.tensorflow.org/tutorials/keras/text_classification
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
'../input/training2/train2', 
batch_size=batch_size, 
validation_split=0.2, 
subset='validation', 
seed=seed)

In [ ]:
#inspired from https://www.tensorflow.org/tutorials/keras/text_classification
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')
max_features = 10000
sequence_length = 500

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)


In [ ]:
#inspired from https://www.tensorflow.org/tutorials/keras/text_classification
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)


In [ ]:
#inspired from https://www.tensorflow.org/tutorials/keras/text_classification
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [ ]:
#inspired from https://github.com/tensorflow/examples/blob/master/community/en/text_classification_solution.ipynb
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))


In [ ]:
#inspired from https://www.tensorflow.org/tutorials/keras/text_classification
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)


In [ ]:
#inspired from https://www.tensorflow.org/tutorials/keras/text_classification
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:

from keras.layers import LSTM,  Dense, Dropout, GlobalMaxPool1D, Bidirectional, Conv1D, MaxPooling1D

embedding_dim = 8


model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.1),
  #layers.Conv1D(filters=256, kernel_size=1),
  #layers.MaxPooling1D(1),
  layers.Conv1D(filters=512, kernel_size=1),
  layers.MaxPooling1D(1),

  layers.Dropout(0.05),
  layers.Bidirectional(LSTM(256, return_sequences = True)),
#  layers.Conv1D(filters=256, kernel_size=1),
#  layers.MaxPooling1D(1),

  #layers.Bidirectional(LSTM(256, return_sequences = True)),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.1),

  layers.Dense(5)])

model.summary()


In [ ]:
#inspired from https://github.com/tensorflow/examples/blob/master/community/en/text_classification_solution.ipynb
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['MeanSquaredError'])



In [ ]:
epochs = 12
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)


In [ ]:
#inspired from https://www.tensorflow.org/tutorials/keras/text_classification
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)


In [ ]:
metadata = pd.read_json('../input/original/test.json', lines=True)
examples = []
example = metadata[['summary']]
exampleReview = metadata[['reviewText']]
for i in range(len(example)):
    examples.append(str(example['summary'][i])+" "+str(exampleReview['reviewText'][i]))
#examples.append(str(example['summary'][1]))

#print(str(example['summary'][0]))
#x=str(examples['summary'][0])
#x=np.array(x)
print(examples)


res = export_model.predict(examples)
#print(res[0])
f = open("basictext.csv", "w")
#f.write("Now the file has more content!")
#f.close()
for i in range (0,10000):
    
    f.write(str(np.argmax(res[i])+1))
    f.write("\n")
#for i in range(len(example)):
#    print(np.argmax(res[i]))